<a href="https://colab.research.google.com/github/mukulre/Data-Science-Projects/blob/main/Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df=pd.read_csv("/content/netflix_content.csv")
df.head()

,Title,Available Globally?,Release Date,Hours Viewed,Language Indicator,Content Type
0,The Night Agent: Season 1,Yes,2023-03-23,"81,21,00,000",English,Show
1,Ginny & Georgia: Season 2,Yes,2023-01-05,"66,51,00,000",English,Show
2,The Glory: Season 1 // 더 글로리: 시즌 1,Yes,2022-12-30,"62,28,00,000",Korean,Show
3,Wednesday: Season 1,Yes,2022-11-23,"50,77,00,000",English,Show
4,Queen Charlotte: A Bridgerton Story,Yes,2023-05-04,"50,30,00,000",English,Movie


In [2]:
df['Hours Viewed']=df['Hours Viewed'].str.replace(',','', regex=False).astype('int64')
df.dropna(subset=['Title'], inplace=True)
df.drop_duplicates(subset=['Title'], inplace=True)

df['Content_ID']=df.reset_index().index.astype('int32')

df['Language_ID']=df['Language Indicator'].astype('category').cat.codes
df['ContentType_ID']=df['Content Type'].astype('category').cat.codes

df[['Content_ID', 'Title','Hours Viewed', 'Language_ID', 'ContentType_ID']].head()

,Content_ID,Title,Hours Viewed,Language_ID,ContentType_ID
0,0,The Night Agent: Season 1,812100000,0,1
1,1,Ginny & Georgia: Season 2,665100000,0,1
2,2,The Glory: Season 1 // 더 글로리: 시즌 1,622800000,3,1
3,3,Wednesday: Season 1,507700000,0,1
4,4,Queen Charlotte: A Bridgerton Story,503000000,0,0


In [3]:
import tensorflow as tf
from tensorflow.keras import layers, Model

num_contents=df['Content_ID'].nunique()
num_languages=df['Language_ID'].nunique()
num_types=df["ContentType_ID"].nunique()

content_input=layers.Input(shape=(1,),dtype=tf.int32, name='content_id')
language_input=layers.Input(shape=(1,),dtype=tf.int32, name='language_id')
type_input=layers.Input(shape=(1,),dtype=tf.int32, name='content_type')

content_embedding=layers.Embedding(input_dim=num_contents+1,output_dim=32)(content_input)
language_embedding=layers.Embedding(input_dim=num_languages+1,output_dim=8)(language_input)
type_embedding=layers.Embedding(input_dim=num_types+1,output_dim=4)(type_input)

content_vec=layers.Flatten()(content_embedding)
language_vec=layers.Flatten()(language_embedding)
type_vec=layers.Flatten()(type_embedding)

combined=layers.Concatenate()([content_vec, language_vec,type_vec])
x=layers.Dense(64,activation='relu')(combined)
x=layers.Dense(32,activation='relu')(x)
output=layers.Dense(num_contents,activation='softmax')(x)

model=Model(inputs=[content_input, language_input, type_input], outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [4]:
model.fit(
    x={
        'content_id':df['Content_ID'],
        'language_id':df['Language_ID'],
        'content_type':df['ContentType_ID']
    },
    y=df['Content_ID'],
    epochs=5,
    batch_size=64
)

Epoch 1/5
300/300 ━━━━━━━━━━━━━━━━━━━━ 13s 38ms/step - accuracy: 0.0000e+00 - loss: 9.8788
Epoch 2/5
300/300 ━━━━━━━━━━━━━━━━━━━━ 12s 39ms/step - accuracy: 0.0000e+00 - loss: 9.8666
Epoch 3/5
300/300 ━━━━━━━━━━━━━━━━━━━━ 21s 39ms/step - accuracy: 9.1716e-04 - loss: 9.6358
Epoch 4/5
300/300 ━━━━━━━━━━━━━━━━━━━━ 22s 46ms/step - accuracy: 0.0105 - loss: 8.2210
Epoch 5/5
300/300 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - accuracy: 0.1075 - loss: 6.2035


In [6]:
import numpy as np
def recommend_similar(content_title, top_k=5):
  content_row=df[df['Title'].str.contains(content_title,case=False,na=False)].iloc[0]
  content_id = content_row['Content_ID']
  language_id = content_row['Language_ID']
  content_type_id = content_row['ContentType_ID']

  predictions = model.predict({
        'content_id': np.array([content_id]),
        'language_id': np.array([language_id]),
        'content_type': np.array([content_type_id])
  })

  top_indices = predictions[0].argsort()[-top_k-1:][::-1]
  recommendations = df[df['Content_ID'].isin(top_indices)]
  return recommendations[['Title', 'Language Indicator', 'Content Type', 'Hours Viewed']]

recommend_similar("Wednesday")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step


,Title,Language Indicator,Content Type,Hours Viewed
3,Wednesday: Season 1,English,Show,507700000
395,Seinfeld: Season 7,English,Show,41700000
614,Shameless (U.S.): Season 8,English,Show,30400000
5294,Club de Cuervos: Season 2,English,Show,2500000
8054,Slam Dunk: Season 8,English,Show,1000000
17023,Samantha!: Season 1,English,Show,100000
